# Simulating Process Flow

# Stages of Manufacturing

## 1. [Raw Material Inventory Store](#first-bullet)

## 2. [Classifier](#second-bullet)

## 3. [Pre-Finish Inventory Store](#third-bullet)

## 4. [Pre-Finish Operation](#fourth-bullet)

## 5. [Pack Inventory Store](#fifth-bullet)

## 6. [Packaging](#sixth-bullet)

![process_flow_diagram](misc/process_flow_diagram.svg)

# TODO:
- **RMI Store**
    - order_drums(): sort drums as dictated by work orders 
- **Classifier**
    - ~~check_pfi_bins(): ensure >5 empty bins available in PFI Store~~
- **PFI Store**
    - assign_drums(): ensure that a given drum maintains the same color & size through a given work order
- **PFO Cell**
    - ~~\_\_repr\_\_(): code proper representation of PFO Cell~~
    - ~~assign_tanks(): determine number of tanks for a given site and initialize them~~
- **PI Cell**
    ~~- assign_drums():~~ 
- **Packaging Cell**
- **Work Orders**
- ~~Finish defining packing cell:~~
    - ~~Assigning machines & properties~~
    - ~~Defining bagging & boxing machines~~
    - ~~Determining process rates~~
- ~~Replace `location` with `plant_id`; update CSVs accordingly~~
- Define how Work Orders are loaded and processed
- Code the overarching simulation process
- Refactoring:
    - ~~Define super/subclass relationship for drums~~
    - ~~Define super/subclass relationship for workcells?~~
    - Pass queue info as dict instead of dataframe
- Thoroughly comment code cells
- Add markdown to illustrate important concepts; break up code cells as necessary
- Add visualizations
- Flowchart Edit Notes:
    - Work orders dictate RMI Store drum order as well as PI Store order.
    - Number and label each work cell for reference in the document.


# 4377 Hours between start & end

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from modules.storage_cell import storage_cell
from modules.processing_cell import processing_cell
from modules.facility import facility

In [170]:
order_bank = pd.read_csv('files/order_bank.csv')

In [173]:
order_bank.groupby(['Color', 'Size', 'Flavor', 'Package Type'], as_index=False)['Qty (pack unit)'].sum()

,Color,Size,Flavor,Package Type,Qty (pack unit)
0,Color Agent1,S1,F1,Bag,24812
1,Color Agent1,S1,F1,Box,2220
2,Color Agent1,S1,F10,Bag,16092
3,Color Agent1,S1,F10,Box,912
4,Color Agent1,S1,F11,Bag,36096
5,Color Agent1,S1,F11,Box,1968
6,Color Agent1,S1,F12,Bag,42504
7,Color Agent1,S1,F12,Box,469
8,Color Agent1,S1,F2,Bag,36532
9,Color Agent1,S1,F3,Bag,25232


In [166]:
detroit = facility('DETROITMI')

Initializing rmi Cell
Initializing classifier Cell
Initializing pfi Cell
Initializing pfo Cell
Initializing pi Cell
Initializing packaging Cell


In [167]:
rmi = detroit.rmi
cfr = detroit.cfr
pfi = detroit.pfi
pfo = detroit.pfo
pis = detroit.pis
pck = detroit.pck

In [169]:
for drum in rmi.drums:
    print(drum.jb_color, drum.contents)

Coloring Agent21 300000
Coloring Agent19 300000
Coloring Agent16 300000
Coloring Agent28 300000
Coloring Agent25 300000
Coloring Agent12 300000
Coloring Agent36 300000
Coloring Agent18 300000
Coloring Agent3 300000
Coloring Agent20 300000
Coloring Agent37 300000
Coloring Agent39 300000
Coloring Agent38 300000
Coloring Agent9 300000
Coloring Agent13 300000
Coloring Agent32 300000
Coloring Agent22 300000
Coloring Agent4 300000
Coloring Agent23 300000
Coloring Agent24 300000
Coloring Agent33 300000
Coloring Agent7 300000
Coloring Agent10 300000
Coloring Agent35 300000
Coloring Agent8 300000
Coloring Agent14 300000
Coloring Agent6 300000
Coloring Agent15 300000
Coloring Agent11 300000
Coloring Agent11 172698
Coloring Agent40 300000
Coloring Agent40 179135
Coloring Agent30 300000
Coloring Agent30 191861
Coloring Agent27 300000
Coloring Agent27 200140
Coloring Agent17 300000
Coloring Agent17 211745
nan 0
nan 0


In [89]:
workorders = pd.read_csv('files/workorder_example.csv')
workorders['unfulfilled'] = True

In [90]:
workorders

,facility,id,jb_color,jb_size,jb_flavor,packaging_type,qty,unfulfilled
0,COLUMBUSOH,21092019,Coloring agent1,S1,F12,Bag,2000,True
1,COLUMBUSOH,21092019,Coloring agent25,S1,F6,Bag,2100,True
2,COLUMBUSOH,21092019,Coloring agent10,S5,F2,Bag,1700,True
3,COLUMBUSOH,21092019,Coloring agent24,S3,F6,Bag,1300,True
4,COLUMBUSOH,21092019,Coloring agent27,S2,F11,Bag,1600,True
5,COLUMBUSOH,21092019,Coloring agent36,S3,F1,Box,250,True
6,COLUMBUSOH,21092019,Coloring agent7,S1,F7,Box,210,True
7,COLUMBUSOH,21092019,Coloring agent8,S4,F10,Box,198,True
8,COLUMBUSOH,21092019,Coloring agent19,S5,F1,Box,214,True
9,COLUMBUSOH,21092019,Coloring agent20,S1,F9,Box,70,True


In [155]:
process_steps = pd.DataFrame()
ship_inventory = pd.DataFrame()
i=0
time=0
flavor = (flavor for flavor in workorders.jb_flavor)
package_type = (package for package in workorders.packaging_type)
pack_qty = (qty for qty in workorders.qty)

while workorders.unfulfilled.count()>0:
    step = {}
    
    if (len(rmi.full_drums)>0) & (len(cfr.avail_mach)>0):
        drum_id, cfr_in = next(rmi.unload_drums())
        step[drum_id+" Empty"]=time
        cfr_id, cfr_time = cfr.load_machines(**cfr_in)
        step[cfr_id+" Load"]=time
        time += cfr_time
        
    if (len(pfi.empty_drums)>4) & (len(cfr.unavail_mach)>0):
        pfi_cap = min([x.capacity for x in pfi.empty_drums])
        machine_id, cfr_out = next(cfr.unload_machines(pfi_cap))
        step[machine_id+" Unload"]=time
        
        drum_ids = pfi.load_drums(cfr_out, time=time)
        for drum_id in drum_ids:
            step[drum_id+" Fill"]=time
    
    if (len(pfi.full_drums)>0) & (len(pfo.avail_mach)>0):
        drum_id, pfo_in = next(pfi.unload_drums())
        step[drum_id+" Empty"]=time
        
        pfo_in['jb_flavor'] = next(flavor)
        pfo_id, pfo_time = pfo.load_machines(**pfo_in)
        step[pfo_id+" Load"]=time
        time += pfo_time
        
    if (len(pis.empty_drums)>0) & (len(pfo.unavail_mach)>0):
        pis_cap = min([x.capacity for x in pis.empty_drums])
        machine_id, pfo_out = next(pfo.unload_machines(pis_cap))
        step[machine_id+" Unload"]=time
        
        drum_ids = pis.load_drums(pfo_out, time=time)
        for drum_id in drum_ids:
            step[drum_id+" Fill"]=time
            
    if (len(pis.full_drums)>0) & (len(pck.avail_mach)>0):
        drum_id, pck_in = next(pis.unload_drums())
        step[drum_id+" Empty"]=time
        
        pck_in['package_type'] = next(package_type)
        pck_id, pck_time = pck.load_machines(**pck_in)
        step[pck_id+" Load"]=time
        time += pck_time
    
    if len(pck.unavail_mach)>0:
        qty = next(pack_qty)
        if pck_in['package_type'] == 'Box':
            cap = 2.5*qty
        else:
            cap = 0.25*qty
        machine_id, pck_out = next(pck.unload_machines(cap))
        pck_out['package_type'] = pck_in['package_type']
        pck_out['qty'] = qty
        step[machine_id+" Unload"]=time
        
        ship_inventory = ship_inventory.append(pck_out, ignore_index=True)
         
    process_steps = process_steps.append(step, ignore_index=True)
    
    i += 1    
    if i == 1000:
        break
    

StopIteration: 

In [165]:
workorders.jb_flavor

0    F12
1     F6
2     F2
3     F6
4    F11
5     F1
6     F7
7    F10
8     F1
9     F9
Name: jb_flavor, dtype: object

In [164]:
next(flavor)

StopIteration: 

In [156]:
ship_inventory

,jb_color,jb_size,jb_flavor,amount,package_type
0,Coloring Agent21,S3,F12,500.0,Bag
1,Coloring Agent21,S3,F12,525.0,Bag
2,Coloring Agent21,S3,F12,425.0,Bag
3,Coloring Agent21,S3,F12,325.0,Bag
4,Coloring Agent21,S3,F12,400.0,Bag
5,Coloring Agent21,S3,F12,62.5,Bag
6,Coloring Agent21,S3,F12,52.5,Bag
7,Coloring Agent21,S3,F12,49.5,Bag
8,Coloring Agent21,S3,F12,53.5,Bag
9,Coloring Agent21,S3,F12,17.5,Bag


In [158]:
fulfillment = ship_inventory.groupby(['jb_color', 'jb_size', 'jb_flavor', 'package_type'], as_index=False)['amount'].count()
fulfillment.rename(columns={'amount':'qty', 'package_type':'packaging_type'}, inplace=True)

In [159]:
fulfillment

,jb_color,jb_size,jb_flavor,packaging_type,qty
0,Coloring Agent21,S3,F12,Bag,10


In [160]:
workorders

,facility,id,jb_color,jb_size,jb_flavor,packaging_type,qty,unfulfilled
0,COLUMBUSOH,21092019,Coloring agent1,S1,F12,Bag,2000,True
1,COLUMBUSOH,21092019,Coloring agent25,S1,F6,Bag,2100,True
2,COLUMBUSOH,21092019,Coloring agent10,S5,F2,Bag,1700,True
3,COLUMBUSOH,21092019,Coloring agent24,S3,F6,Bag,1300,True
4,COLUMBUSOH,21092019,Coloring agent27,S2,F11,Bag,1600,True
5,COLUMBUSOH,21092019,Coloring agent36,S3,F1,Box,250,True
6,COLUMBUSOH,21092019,Coloring agent7,S1,F7,Box,210,True
7,COLUMBUSOH,21092019,Coloring agent8,S4,F10,Box,198,True
8,COLUMBUSOH,21092019,Coloring agent19,S5,F1,Box,214,True
9,COLUMBUSOH,21092019,Coloring agent20,S1,F9,Box,70,True


In [161]:
len(fulfillment.merge(workorders[['jb_color','jb_size', 'jb_flavor', 'packaging_type', 'qty']])) == len(fulfillment)

False

In [162]:
process_steps

,PFI Drum1 Empty,PFI Drum1 Fill,PFI Drum2 Fill,PFI Drum3 Fill,PFI Drum4 Fill,PFI Drum5 Fill,PI Drum 1 Empty,PI Drum 1 Fill,RMI DRUM1 Empty,boxing_machine1 Load,...,PI Drum 3 Fill,PFI Drum14 Fill,PFI Drum15 Fill,PI Drum 4 Fill,PI Drum 5 Fill,PI Drum 6 Fill,PFI Drum4 Empty,PI Drum 7 Fill,PFI Drum5 Empty,PI Drum 8 Fill
0,87.719298,87.719298,87.719298,87.719298,87.719298,87.719298,95.123934,95.123934,0.0,95.123934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100.010199,100.010199,NaN,NaN,NaN,NaN,107.001804,107.001804,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110.262082,110.262082,NaN,NaN,NaN,NaN,NaN,117.591094,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,134.537253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,134.537253,134.537253,134.537253,134.537253,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,134.537253,134.537253,141.127722,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,148.030215,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,154.767955,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,154.767955,161.222762,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.222762,168.870867


In [163]:
process_steps.to_csv('process_steps.csv')

## 1. Raw Material Inventory (RMI) Store <a class="anchor" id="first-bullet"></a>
Several drums act as an inventory store for raw material. The number of drums and their capacities varies by site. Drums can only hold jelly beans with the same color (many-to-one relationship between drums and jelly beans of a given color). One RMI drum can be released at a time into the classifier.

Rules:
1. Order of release from RMI drums is dictated by workorders
2. If there are two RMI drums with the same color at a given location, start emptying the drum with the lower equipment number first
3. Releasing jelly beans from a drum is only allowed if there are at least 5 pre-finished inventory bins empty and available (the next step, classification, is a continuous process which fills pre-finished inventory bins)

In [ ]:
detroit_rmi = storage_cell('rmi', num_drums=40, facility='DETROITMI')

In [ ]:
detroit_rmi.drums

In [ ]:
detroit_rmi.full_drums

In [ ]:
detroit_rmi.empty_drums

In [ ]:
pre_prod_move = pd.DataFrame(
    {
        'Color':['Coloring Agent5', 'Coloring Agent2'],
        'Rem':[176427, 161034]
    }
)

In [ ]:
detroit_rmi.fill_drums(pre_prod_move)

In [ ]:
detroit_rmi.full_drums

In [ ]:
detroit_rmi.empty_drums

In [ ]:
detroit_rmi.fill_drums(pre_prod_move)

## 2. Classifier <a class="anchor" id="second-bullet"></a>

The RMI drums contain jelly beans of the same color, but of varying size. RMI drums are emptied one by one into the classifier, which sorts jelly beans into 5 distinct sizes. Percentage split of each jelly bean size for a given color is provided with the problem statement.

In [ ]:
detroit_classifier = processing_cell('classifier', num_machines=1, facility='DETROITMI')

In [ ]:
detroit_classifier.load_machines(300000, jb_color='Coloring Agent1')

In [ ]:
detroit_classifier.empty_machines(10000)

In [ ]:
detroit_classifier.machines

In [ ]:
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

## 3. Pre-Finish Inventory (PFI) Store <a class="anchor" id="third-bullet"></a>

Jelly beans sorted by the classifier are stored in PFI drums. Drums are assigned a specific jelly bean size and color to avoid contamination (the PFI drums are reused once the drums are emptied into the Pre-Finish Operation). As mentioned earlier, 5 PFI drums (one for each size) must be empty before an RMI drum can be unloaded (thus starting the classification process).

Rules:
1. The classification process simultaneously splits the jelly beans according to the given ratios directly into the PFI drums
2. Fundamentally any PFI drum can be filled with any size of jelly bean; however, once the assignment is made for a given work order, the PFI drum must only hand jelly beans of that size & color until the work order is completely processed
3. PFI drums can only be filled to 95% capacity
4. PFI drums are only available to be emptied into the Pre-Finish Operation once filling has stopped

In [ ]:
detroit_pfi = storage_cell('pfi', num_drums=15, facility='DETROITMI')
detroit_pfi.drums

In [ ]:
detroit_pfi.empty_drums

In [ ]:
detroit_pfi.drums[0].capacity

In [ ]:
detroit_pfi.drums[0].contents

In [ ]:
detroit_pfi.fill_drums(detroit_classifier.empty_machines(10000))

In [ ]:
detroit_pfi.drums[0].contents

In [ ]:
detroit_pfi.drums[0].jb_size

In [ ]:
detroit_pfi.drums[0].jb_color

In [ ]:
detroit_pfi.full_drums

In [ ]:
detroit_pfi.empty_drums

## 4. Pre-Finish Operation (PFO) <a class="anchor" id="fourth-bullet"></a>

PFI drums are now emptied into a tank which applies flavoring to jelly beans of a given color and size (i.e., the tank can only hold 1 unique color-size-flavor combinations at any given time). The number of tanks, tank capacity, and processing rate are given for each facility as part of the problem statement.

Rules:
1. If there is more than 1 PFO tank, assume the rate for each tank is the same.
2. When flavors are changed, there is a change-over duration of 5 minutes.
3. There is no change-over duration between size changes.

In [ ]:
detroit_pfo = processing_cell('pfo', num_machines=2, facility='DETROITMI')

In [ ]:
detroit_pfo

In [ ]:
detroit_pfo.machines

In [ ]:
detroit_pfo.load_machines(10000, jb_color='Coloring Agent1', jb_size='S1', jb_flavor='F1')

In [ ]:
pfo0 = detroit_pfo.machines[0]

In [ ]:
pfo0.available

In [ ]:
pfo0.queue

In [ ]:
pfo0.unload(10000)

In [ ]:
pfo0.jb_color

In [ ]:
pfo0.available

## 5. Pack Inventory (PI) Store <a class="anchor" id="fourth-bullet"></a>

Once the PFO is complete, flavored jelly beans are staged in PI drums. Each PI drum contains fully differentiated product: jelly beans grouped by color, size, and flavor. The number of drums and their capacities vary by site and are given within the problem statement.

Rules:
1. Release of jelly beans into the PFO follows a 'FIFO' policy by default (other policies are allowed but must be explicitly specified).
2. Jelly beans flow continuously from the PFI store through the PFO into the PI store. It can be assumed that any PFO can feed any PI drum.
3. To avoid overflow, PI drums are filled to 95%.
4. PI drums can only be emptied into the Packaging operation once they have been disengaged from the PFO.
5. The lowest PI Drum number is filled up first.
6. Jelly bean colors, flavors, and sizes cannot be mixed in a given drum.
7. Only one PI drum can be emptied into the Packaging operation at a time.
8. The quantity of jelly beans released into the Packaging operation will be determined by the Work Orders.

In [ ]:
detroit_pi = storage_cell('pi', num_drums=8, facility='DETROITMI')

In [ ]:
detroit_pi.drums

In [ ]:
drum = detroit_pi.drums[1]
drum.contents

In [ ]:
drum.capacity

In [ ]:
detroit_pi.full_drums

In [ ]:
detroit_pi.empty_drums

## 6. Packaging Operation <a class="anchor" id="sixth-bullet"></a>

PI drums are emptied for packaging either a bag or a box; i.e., a drum can only be emptied to feed a bag or a box.

Rules:
1. The bagging and box lines cannot be run simultaneously.
2. The box line takes precedence over the bagging line.
3. Orders for boxes are fulfilled prior to orders for bags.
4. All excess material is to be stored in bags.

In [ ]:
detroit_packaging = processing_cell(
    'packaging',
    facility='DETROITMI',
    bagging_machines=1, 
    boxing_machines=1
)

In [ ]:
detroit_packaging.machines

In [ ]:
detroit_packaging.load_machines(
    10000, 
    jb_color='Coloring Agent1', 
    jb_size='S1',
    packaging_type='box'
)

In [ ]:
detroit_packaging.empty_machines(2500)

## Workorder Processing

## Testing & Validation

### Facility Initialization